In [1]:
from bs4 import BeautifulSoup
import re
import requests
from tqdm import tqdm

/Users/rajibdas/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
questions = []
answers = []

In [3]:
web = "https://artofproblemsolving.com"
url = "https://artofproblemsolving.com/wiki/index.php/AIME_Problems_and_Solutions"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

td_tags = soup.find_all('td')
hrefs = [a['href'] for td in td_tags for a in td.find_all('a')]

def extract_problem_text(soup):
    parts = []
    problem_heading = soup.find('span', id=re.compile(r'^Problem(_\d+)?(.+)?$')).parent
    for element in problem_heading.find_next_siblings():
        if element.name == 'h2':
            break
        if element.name == 'p':
            for child in element.descendants:
                if child.name == 'img' and 'latex' in child.get('class', []):
                    parts.append(child.get('alt', ''))
                elif isinstance(child, str):
                    parts.append(child.strip())
    return ' '.join(parts)

def extract_solution_text(soup):
    parts = []
    solution_heading = soup.find('span', id=re.compile(r'^Solution(_\d+)?(.+)?$')).parent
    for element in solution_heading.find_all_next():
        if element.name == 'h2':
            break
        if element.name == 'p':
            for child in element.descendants:
                if child.name == 'img' and 'latex' in child.get('class', []):
                    parts.append(child.get('alt', ''))
                elif isinstance(child, str):
                    parts.append(child.strip())
    return ' '.join(parts)


for link in tqdm(hrefs):
    new_link = web+link
    response = requests.get(new_link)
    soup = BeautifulSoup(response.content, 'html.parser')
    q = soup.find_all('td')[-2]
    qls = [a['href'] for a in q.find_all('a')]
    for ql in qls:
        q_link = web + ql
        response = requests.get(q_link)
        soup = BeautifulSoup(response.content, 'html.parser')
        p = extract_problem_text(soup)
        a = extract_solution_text(soup)
        
        questions.append(p)
        answers.append(a)

100%|██████████| 67/67 [12:10<00:00, 10.90s/it]


In [4]:
import pandas as pd
df = pd.DataFrame({"questions": questions, "answers": answers})

In [5]:
df.to_csv("aime.csv")